## Formula1 Data loading
- Ingest four tables in to Bronze schema add ingestion date column

In [0]:
from pyspark.sql.functions import *

In [0]:
catalog = "cgi_dev"
bronze_schema = "navaneeth_bronze"
silver_schema = "navaneeth_silver"

file_path_drivers="/Volumes/cgi_dev/naval/formula1/drivers.json"
file_path_constructors = "/Volumes/cgi_dev/naval/formula1/constructors.json"
file_path_pit_stops = "/Volumes/cgi_dev/naval/formula1/pit_stops.json"
file_path_circuts= "/Volumes/cgi_dev/naval/formula1/circuits.csv"

Function to Load JSON file 

In [0]:
def read_write_json(file_path, bronze_catalog, bronze_schema, bronze_table_name):
    #print(bronze_table_name)
    df = spark.read.option("multiline","true").json(file_path).withColumn("ingestion_datetime", current_timestamp()).withColumn("path", col("_metadata"))
    df.write.mode("overwrite").saveAsTable(f"{bronze_catalog}.{bronze_schema}.{bronze_table_name}")
    

In [0]:
read_write_json(file_path_drivers, catalog, bronze_schema, "drivers")
read_write_json(file_path_constructors, catalog, bronze_schema, "constructors")
read_write_json(file_path_pit_stops, catalog, bronze_schema, "pitstops")

Load CSV file

In [0]:
df_circuits = spark.read.csv(file_path_circuts, inferSchema=True, header=True).withColumn("ingestion_datetime", current_timestamp()).withColumn("path", col("_metadata"))
df_circuits.write.mode("overwrite").saveAsTable(f"{catalog}.{bronze_schema}.circuits")

Check the record counts in tables

In [0]:
%sql
SELECT "circuts" AS table_name, COUNT(1) AS record_count 
FROM cgi_dev.navaneeth_bronze.circuts
UNION ALL
SELECT "drivers" AS table_name, COUNT(1) AS record_count 
FROM cgi_dev.navaneeth_bronze.drivers
UNION ALL
SELECT "constructors" AS table_name, COUNT(1) AS record_count 
FROM cgi_dev.navaneeth_bronze.constructors
UNION ALL
SELECT "pitstops" AS table_name, COUNT(1) AS record_count 
FROM cgi_dev.navaneeth_bronze.pitstops

In [0]:
%sql
Select *,
name.forename as forename,
name.surname as surname
 from cgi_dev.navaneeth_bronze.drivers

In [0]:
df_drivers = spark.table(f"{catalog}.{bronze_schema}.drivers")\
.withColumn("forename",col("name.forename"))\
.withColumn("surname",col("name.surname"))\
  .drop("name","path","ingestion_datetime")

df_drivers.write.mode("overwrite").saveAsTable(f"{catalog}.{silver_schema}.drivers")
